In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine
import psycopg2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### criar script para unir arquivos excel

In [ ]:
base_folder = '/content/drive/MyDrive/sample_data/Finance_Data'

dfs = []


for year_folder in os.listdir(base_folder):
  year_path = os.path.join(base_folder,year_folder)

  if os.path.isdir(year_path):
    for file in os.listdir(year_path):
      if file.endswith('.xlsx'):
        file_path = os.path.join(year_path,file)
        df = pd.read_excel(file_path)
        dfs.append(df)

In [ ]:
dfs

[         DATE      LOCATION           ITEM    COST  TOTAL  \
 0  2020-04-01       Safeway           Food   -5.37    NaN   
 1  2020-04-01       Shirley           Rent -670.00    NaN   
 2  2020-04-01     Starbucks         Coffee   -4.67    NaN   
 3  2020-04-03       Safeway           Food   -5.37    NaN   
 4  2020-04-03     Starbucks         Coffee   -4.67    NaN   
 5  2020-04-05       Safeway           Food  -25.56    NaN   
 6  2020-04-05  London Drugs  Entertainment  -78.74    NaN   
 7  2020-04-06     Starbucks         Coffee   -5.20    NaN   
 8  2020-04-06       Safeway           Food   -5.34    NaN   
 9  2020-04-07       Safeway           Food  -13.33    NaN   
 10 2020-04-08       Safeway           Food   -5.34    NaN   
 11 2020-04-08     Starbucks         Coffee   -5.20    NaN   
 12 2020-04-09     Starbucks         Coffee   -4.67    NaN   
 13 2020-04-09       Safeway           Food   -5.34    NaN   
 14 2020-04-10       Safeway           Food   -5.37    NaN   
 15 2020

In [ ]:
df_final = pd.concat(dfs,ignore_index=True)

In [ ]:
df_final

,DATE,LOCATION,ITEM,COST,TOTAL,COMMENTS
0,2020-04-01 00:00:00,Safeway,Food,-5.37,NaN,NaN
1,2020-04-01 00:00:00,Shirley,Rent,-670.00,NaN,NaN
2,2020-04-01 00:00:00,Starbucks,Coffee,-4.67,NaN,NaN
3,2020-04-03 00:00:00,Safeway,Food,-5.37,NaN,NaN
4,2020-04-03 00:00:00,Starbucks,Coffee,-4.67,NaN,NaN
...,...,...,...,...,...,...
1062,2019-08-29 00:00:00,Safeway,Food,-11.99,NaN,Grandma had her hip surgery today
1063,2019-08-29 00:00:00,Peter Lougheed Centre,Restaurant,-13.70,NaN,NaN
1064,2019-08-30 00:00:00,Peter Lougheed Centre,Restaurant,-18.20,NaN,NaN
1065,2019-08-31 00:00:00,Subway,Restaurant,-10.80,NaN,NaN


In [ ]:
df = df_final

In [ ]:
df.to_csv('finance_data.csv',index=False)

### bronze

In [ ]:
user = 'colab'
password = 'atip'
endpoint = 'atip-learn-01.cl2y2k0oopwh.us-east-1.rds.amazonaws.com'
port = '5432'
database = 'atiplearn'

In [ ]:
conn = create_engine(f'postgresql+psycopg2://{user}:{password}@{endpoint}:{port}/{database}')

In [ ]:
df.to_sql('finance_data',con=conn,if_exists='replace',schema='bronze',index=False)

67

## silver

In [ ]:
query = '''select "DATE","LOCATION","ITEM","COST","COMMENTS"  from bronze.finance_data '''
df_silver = pd.read_sql(query,con=conn)
df_silver.head(1)

,DATE,LOCATION,ITEM,COST,COMMENTS
0,2020-04-01 00:00:00,Safeway,Food,-5.37,None


In [ ]:
df_silver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DATE      1067 non-null   object 
 1   LOCATION  1067 non-null   object 
 2   ITEM      1067 non-null   object 
 3   COST      1067 non-null   float64
 4   COMMENTS  162 non-null    object 
dtypes: float64(1), object(4)
memory usage: 41.8+ KB


In [ ]:
df_silver['DATE'] = pd.to_datetime(df_silver['DATE'],format='mixed',errors='coerce')

In [ ]:
df_silver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   DATE      1064 non-null   datetime64[ns]
 1   LOCATION  1067 non-null   object        
 2   ITEM      1067 non-null   object        
 3   COST      1067 non-null   float64       
 4   COMMENTS  162 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 41.8+ KB


In [ ]:
nulos = df_silver[df_silver['DATE'].isnull()]
nulos

,DATE,LOCATION,ITEM,COST,COMMENTS


In [ ]:
df_silver['DATE'] = df_silver['DATE'].fillna(method='bfill')

<ipython-input-32-1ea3c016088a>:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_silver['DATE'] = df_silver['DATE'].fillna(method='bfill')


In [ ]:
df_silver[df_silver.duplicated()]

,DATE,LOCATION,ITEM,COST,COMMENTS
614,2019-04-22,Subway,Restaurant,-9.96,None
615,2019-04-23,Subway,Restaurant,-9.96,None
616,2019-04-24,Subway,Restaurant,-9.96,None
621,2019-04-25,Safeway,Food,-6.06,None
812,2019-10-22,Korean BBQ,Restaurant,-11.50,None


df_silver = df_silver.drop_duplicates()

In [ ]:
df_silver.to_sql('finance_data',con=conn,if_exists='replace',schema='silver',index=False)

67

### Gold

In [ ]:
df_gold = pd.read_sql('select * from silver.finance_data',con=conn)
df_gold.head(1)

,DATE,LOCATION,ITEM,COST,COMMENTS
0,2020-04-01,Safeway,Food,-5.37,None


In [ ]:
df_gold['COST'] = df_gold['COST']*(-1)
df_gold

,DATE,LOCATION,ITEM,COST,COMMENTS
0,2020-04-01,Safeway,Food,5.37,None
1,2020-04-01,Shirley,Rent,670.00,None
2,2020-04-01,Starbucks,Coffee,4.67,None
3,2020-04-03,Safeway,Food,5.37,None
4,2020-04-03,Starbucks,Coffee,4.67,None
...,...,...,...,...,...
1062,2019-08-29,Safeway,Food,11.99,Grandma had her hip surgery today
1063,2019-08-29,Peter Lougheed Centre,Restaurant,13.70,None
1064,2019-08-30,Peter Lougheed Centre,Restaurant,18.20,None
1065,2019-08-31,Subway,Restaurant,10.80,None


In [ ]:
df_gold.to_sql('finance_data',con=conn,if_exists='replace',schema='gold',index=False)

67